<a href="https://colab.research.google.com/github/allengongerr/artifical_intelligence/blob/main/Gong_Allen_assignment_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Artificial Intelligence
# 464/664
# Assignment #8

## General Directions for this Assignment

00. We're using a Jupyter Notebook environment (tutorial available here: https://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html),
01. Output format should be exactly as requested (it is your responsibility to make sure notebook looks as expected on Gradescope),
02. Check submission deadline on Gradescope,
03. Rename the file to Last_First_assignment_8,
04. Submit your notebook (as .ipynb, not PDF) using Gradescope, and
05. Do not submit any other files.

## Before You Submit...

1. Re-read the general instructions provided above, and
2. Hit "Kernel"->"Restart & Run All".

## Language Modeling

This homework will require you to load and train models.  If you choose small models and datasets, you should be able to run this locally on your computer. However, larger models/datasets may require GPU access. You can access one GPU for free on [Google Colab](https://colab.research.google.com/).

We will use HuggingFace libraries in this assignment. We discussed majority of what you will need during the discussion demo. Additional documentation can be found [here](https://huggingface.co/docs).

In [1]:
!pip install evaluate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
# Imports
import torch
import numpy as np
import evaluate
from datasets import load_dataset
from transformers import pipeline
from transformers import DataCollatorWithPadding
from transformers import TrainingArguments, Trainer
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification, AutoModelForCausalLM
import os
os.environ["WANDB_DISABLED"] = "true"

## **IMPORTANT NOTE**

I did all my work in Google Colab then downloaded it as a .ipynb file so I could submit this assignment to gradescope. On Colab, I clicked "restart session and run all," which I assumed to be the equivalent of "Restart & Run All" for the kernel.


## Problem 0: Data
From the [HuggingFace Datasets](https://huggingface.co/datasets), choose a dataset that satisfies the following criteria:
- Data must have train and test splits (Optional development set)
- Task must be text classification
- Task must have at least 3 labels

In [3]:
# Load the data here
dataset = load_dataset("ag_news")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/8.07k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/18.6M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

**Describe the data.**
What is the utility of the task? What are the inputs? What are the labels? Are the any potential difficulties you expect from the task? How do you evaluate the performance of this task?

TODO

The AG News dataset is a collection of news articles gathered from more than 2000 news sources. Each data point usually consists of a short piece of text (the news article title and sometimes a snippet of the news article) along with a corresponding news category label. The main task with AG News is text classification—specifically, classifying a piece of text into one of several predefined news categories.

**Inputs and Labels:**
Each example typically contains a title and description (or short snippet) of a news article.
While some approaches use only the title, most leverage both the title and the description as the full textual input. There are four classes in this dataset, each representing broad topical categories: World, Sports, Business, Sci/Tech

**Potential Difficulties:**
1.   Ambiguity for shorter texts ca be a problem since article snippetrs sometimes are brief and don't provide much context
2.   Overlap between the categories can sometimes be similar or related
3. There could be subtle biases depending on the sources of the news articles

**Performance Evaluation:**

Common evaluation metrics include:

*   Accuracy: The percentage of correctly classified articles
*   Precision, Recall, F1-Score (per class and macro-averaged): Useful when you want a more detailed understanding of performance across different classes, especially if class distribution is somewhat imbalanced
*   Confusion Matrix: Helps visualize which categories are often mistaken for one another.

Accuracy is often good enough for  a balanced dataset, but using macro-F1 can give a better sense of performance equality across all categories.








**Research current methods using this dataset.**
What is the current state of the art method? Describe the method, including the type of model used, training protocol (if any), and the performance. Cite your sources.

The current state of the art method, according to [Papers from Code's AG News](https://paperswithcode.com/sota/text-classification-on-ag-news) and other relevant research publications, top results often come from large, pretrained Transformer-based language models. Among these, methods like Charformer, DeBERTa, and other advanced Transformer variants have been reported to achieve very high accuracy on AG News (often above 95%). For example, Charformer (Hix-128 variant), introduced by [Tay et al. (2021)](https://arxiv.org/abs/2106.12672), has been cited for achieving strong results on the AG News benchmark. Charformer uses a novel gradient-based subword tokenization approach at the character level, which allows it to efficiently learn representations without relying on predefined subwords. This leads to strong performance across a variety of text classification tasks, including AG News.

Types of models include large pre-trainined language models and models with transformer-based architectucres such as Charformer, BERT, and DeBERTa.

**Method Description**: The Charformer operates on characters instead of subwords or words. It uses a novel gradient-based subword tokenization approach to learn how to form token representations. By learning tokenization during the training process, the Charformer can learn robust patterns at the character-level in an adaptive manner. These types of models are first pretrained on large corpora, and after pretraining, it's finetuned on the AG News dataset. During fine-tuning, the goal is to minimize classficiation loss and correctly predict the news category. Parameters such as learning rate, batch size, and number of epochs are tuned for the best performance.

**Performance**: the best transformer models are known to achieve above a 95% accuracy score no AG News. Charformer's reported accuracy is around 95.3-95.6%.

**Sources**:

Tay, Y., Metzler, D., Zhao, Z., & Bahdanau, D. (2021). Charformer: Fast Character Transformers via Gradient-based Subword Tokenization. [ArXiv:2106.12672](https://ArXiv:2106.12672)

Papers with Code: AG News Classification Leaderboard: https://paperswithcode.com/sota/text-classification-on-ag-news

TODO

(Optional) If necessary, perform any data preprocessing here. For example, depending on the dataset you choose, you may need to clean the text or split the training set into a train and validation set.

In [4]:
# TODO: Dataset preprocessing (Optional)

## Problem 1: Encoder only models
Choose an encoder only model (e.g. BERT). Load the model and add a classification layer.

Describe the model you choose. What are the unique properties of this model? What are the pros and cons? Cite your sources.

I chose the BERT (Bidirectional Encoder Representations from Transformers) model. It uses a bidirectional attention mechanism, considering both the left and right context of each token. This enables the model to build deeply contextualized representations of language. BERT was pre-trained on large corpora (such as Wikipedia and BooksCorpus) using two self-supervised objectives:

1. Masked Language Modeling (MLM): Randomly masks some input tokens and trains the model to predict the masked tokens.

2. Next Sentence Prediction (NSP): Trains the model on pairs of sentences, predicting whether the second sentence indeed follows the first, thus helping BERT learn sentence-level relationships.

**Unique Properties**
* Bidirectional Contextualization:
Unlike unidirectional models, BERT uses a bidirectional attention mechanism that considers context from both the left and right of a given token. This leads to rich, context-aware embeddings.

* Pretraining-Finetuning Paradigm:
BERT introduces a framework where a large, general-purpose language model is pre-trained on massive unlabeled data, and then fine-tuned on much smaller task-specific datasets.

* Versatility and Transferability:
BERT representations can be adapted to many downstream tasks—classification, question answering, named entity recognition—with minimal changes to the architecture

Pros:
* Strong Baseline Performance:
BERT set new performance benchmarks on a variety of NLP tasks upon release, often improving state-of-the-art by substantial margins.

* Reduced Task-Specific Data Requirements:
Because BERT is extensively pre-trained, it typically requires less labeled data than models trained from scratch to achieve good performance.

* General-Purpose Language Understanding:
BERT’s representations are useful as general embeddings for a wide range of tasks, making it a versatile component of modern NLP pipelines.

Cons:
* Computationally Expensive:
Pre-training and fine-tuning BERT involves significant computational resources. Even at inference time, BERT can be slower compared to smaller, more specialized models.

* Model Size and Latency:
Models like bert-base (~110M parameters) or bert-large (~340M parameters) can be large, increasing memory footprint and inference latency.

* Context Window Limitations:
Standard BERT models are limited to inputs of a certain maximum length (often 512 tokens). Longer inputs must be truncated or split, potentially losing context.

**Sources**:

[Tech Target Article](https://www.techtarget.com/searchenterpriseai/definition/BERT-language-model#:~:text=What%20is%20BERT%3F,surrounding%20text%20to%20establish%20context.)

Devlin, J., Chang, M.-W., Lee, K., & Toutanova, K. (2018). BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding. [arXiv:1810.04805]()

Wolf, T., et al. (2020). Transformers: State-of-the-Art Natural Language Processing. In Proceedings of EMNLP 2020. [arXiv:1910.03771]()

TODO

Finetune the model on your dataset. Report the performance on the test set.

In [5]:
# TODO
from transformers import BertTokenizerFast, BertConfig, BertModel, Trainer, TrainingArguments
import torch
import torch.nn as nn

model_name = "bert-base-uncased"
tokenizer = BertTokenizerFast.from_pretrained(model_name)

class BertClassifier(nn.Module):
    def __init__(self, model_name, num_labels=4):
        super(BertClassifier, self).__init__()
        self.config = BertConfig.from_pretrained(model_name)
        self.bert = BertModel.from_pretrained(model_name)
        self.dropout = nn.Dropout(self.config.hidden_dropout_prob)
        self.classifier = nn.Linear(self.config.hidden_size, num_labels)

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output  # [batch_size, hidden_size]
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)  # [batch_size, num_labels]

        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits, labels)
        return {"loss": loss, "logits": logits}


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Model Initialization and Finetuning

In [6]:
model = BertClassifier(model_name, num_labels=4)
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=128)

encoded_dataset = dataset.map(preprocess_function, batched=True)
encoded_dataset = encoded_dataset.remove_columns(["text"])
encoded_dataset = encoded_dataset.rename_column("label", "labels")
encoded_dataset.set_format("torch")

# the dataset is super large, so I will take a small sample size
train_dataset = encoded_dataset["train"].select(range(2000))
test_dataset = encoded_dataset["test"].select(range(500))
accuracy_metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return accuracy_metric.compute(predictions=predictions, references=labels)

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    logging_steps=500,
    save_steps=500,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    logging_dir="./logs"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

trainer.train()
bert_eval_results = trainer.predict(test_dataset)
print("BERT Test Set Performance:", bert_eval_results.metrics)
bert_preds = np.argmax(bert_eval_results.predictions, axis=-1)
refs = bert_eval_results.label_ids


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Map:   0%|          | 0/120000 [00:00<?, ? examples/s]

Map:   0%|          | 0/7600 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.331764,0.902000
2,No log,0.331161,0.884000


BERT Test Set Performance: {'test_loss': 0.3317643702030182, 'test_accuracy': 0.902, 'test_runtime': 3.4212, 'test_samples_per_second': 146.149, 'test_steps_per_second': 9.354}


## Problem 2: Decoder only models
Choose an decoder only model (e.g. GPT2). Describe the model you choose. What are the unique properties of this model? What are the pros and cons? Cite your sources.

For this problem, I chose GPT-2, a decoder-only model from the Transformer family developed by OpenAI. GPT-2 is designed to generate text by predicting the next word in a sequence based on the previous context. Unlike models like BERT, which are encoder-based and focus on understanding text for tasks like classification, GPT-2 only uses a decoder. This makes it highly effective for generative tasks, such as text generation and language modeling.

**Unique Properties**: One of GPT-2's unique features is that it is autoregressive, meaning it generates one token at a time in sequence. The model leverages multi-layer self-attention and positional embeddings to process input tokens and capture long-range dependencies in text. GPT-2 was pretrained on massive amounts of text data, giving it a strong ability to generalize to various natural language tasks without task-specific training.

Pros:
* GPT-2 performs very well for text generation, summarization, and other generative tasks.
* Since it is pretrained on a large corpus, it requires minimal fine-tuning for downstream tasks.
* It can handle diverse styles of text and maintain coherent context over long passages.

Cons:
* GPT-2 lacks bidirectional understanding since it only processes input from left to right. This can make it less effective for tasks that require a complete understanding of the entire input (e.g., text classification).
* It is computationally expensive to train and deploy due to its large size.
* The model has a tendency to produce repetitive or nonsensical outputs without careful prompt engineering.

**Sources**:
Radford, A., Wu, J., Child, R., Luan, D., Amodei, D., & Sutskever, I. (2019). Language Models are Unsupervised Multitask Learners. OpenAI Blog. https://cdn.openai.com/better-language-models/language_models_are_unsupervised_multitask_learners.pdf

Hugging Face GPT-2 Documentation: [https://huggingface.co/gpt2]()

TODO

Load the model and use prompting for your task. You will likely need to write a helper function to parse the answer.

(Ex. “The answer is 1” -> 1). Report the performance on the test set.

In [7]:
# TODO
from transformers import GPT2Tokenizer, GPT2LMHeadModel

gpt2_test_raw = dataset["test"].select(range(500))
texts = gpt2_test_raw["text"]  # original texts for error analysis

model_name = "gpt2"
gpt2_tokenizer = GPT2Tokenizer.from_pretrained(model_name)
gpt2_model = GPT2LMHeadModel.from_pretrained(model_name)

device = "cuda" if torch.cuda.is_available() else "cpu"
gpt2_model.to(device)
gpt2_model.eval()

label_map = {
    0: "World",
    1: "Sports",
    2: "Business",
    3: "Sci/Tech"
}

def parse_answer(output_text):
    key_phrase = "The answer is"
    start_idx = output_text.find(key_phrase)
    if start_idx == -1:
        return None
    start_idx += len(key_phrase)
    answer_part = output_text[start_idx:].strip()
    if len(answer_part) > 0 and answer_part[0].isdigit():
        return int(answer_part[0])
    else:
        return None

def predict_label_gpt2(text):
    prompt = (
        f"Categories:\n"
        f"0) World\n"
        f"1) Sports\n"
        f"2) Business\n"
        f"3) Sci/Tech\n\n"
        f"Given the following news article, classify it by choosing a number:\n"
        f"\"{text}\"\n"
        f"The answer is"
    )
    inputs = gpt2_tokenizer(prompt, return_tensors="pt").to(device)
    output_ids = gpt2_model.generate(
        **inputs,
        max_length=inputs["input_ids"].shape[1] + 5,
        num_return_sequences=1,
        do_sample=False,
        pad_token_id=gpt2_tokenizer.eos_token_id
    )
    output_text = gpt2_tokenizer.decode(output_ids[0], skip_special_tokens=True)
    predicted_label = parse_answer(output_text)
    if predicted_label is None:
        predicted_label = 0
    return predicted_label

gpt2_preds = []
for text in texts:
    gpt2_preds.append(predict_label_gpt2(text))

gpt2_results = accuracy_metric.compute(predictions=gpt2_preds, references=refs)
print("GPT-2 Test Set Performance:", gpt2_results)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

GPT-2 Test Set Performance: {'accuracy': 0.244}


## Problem 3: Error Analysis

Conduct an error analysis on both models. What is each model good at? What do they get wrong? Provide examples of both correct and incorrect predictions. Suggest methods to improve the performance.

In [8]:
# TODO
import random

def get_examples(texts, refs, preds):
    correct = []
    incorrect = []
    for i, (pred, ref) in enumerate(zip(preds, refs)):
        if pred == ref:
            correct.append((texts[i], label_map[ref], label_map[pred]))
        else:
            incorrect.append((texts[i], label_map[ref], label_map[pred]))
    return correct, incorrect

bert_correct, bert_incorrect = get_examples(texts, refs, bert_preds)
gpt2_correct, gpt2_incorrect = get_examples(texts, refs, gpt2_preds)

def print_examples(examples, title, num=2):
    print(title)
    for ex in random.sample(examples, min(num, len(examples))):
        text, true_label, pred_label = ex
        print(f"Text: {text}")
        print(f"True Label: {true_label}, Predicted Label: {pred_label}\n")

print("==== BERT Error Analysis ====")
print_examples(bert_correct, "BERT Correct Predictions:")
print_examples(bert_incorrect, "BERT Incorrect Predictions:")

print("==== GPT-2 Error Analysis ====")
print_examples(gpt2_correct, "GPT-2 Correct Predictions:")
print_examples(gpt2_incorrect, "GPT-2 Incorrect Predictions:")

==== BERT Error Analysis ====
BERT Correct Predictions:
Text: Admirers look to 2008 But as far as swim greats Rowdy Gaines and John Naber are concerned, what Phelps did in Athens exceeded what Spitz did in Munich in 1972.
True Label: Sports, Predicted Label: Sports

Text: Sutton Adds Haas, Cink to Ryder Cup Team  MILWAUKEE (Sports Network) - U.S. Ryder Cup captain Hal  Sutton finalized his team on Monday when he announced the  selections of Jay Haas and Stewart Cink as his captain's picks.
True Label: Sports, Predicted Label: Sports

BERT Incorrect Predictions:
Text: 1994 Law Designed to Preserve Guard Jobs (AP) AP - A 1994 law strengthened job protections for National Guard and Reserve troops called to active duty. Here are major provisions of the Uniformed Services Employment and Reemployment Rights Act (USERRA).
True Label: World, Predicted Label: Sci/Tech

Text: Audit finds no fraud in Venezuelan recall vote CARACAS : An audit of last Sunday #39;s recall vote in Venezuela, which fa

When we look at where the BERT and GPT-2 models succeed or stumble on AG News articles, some patterns stand out. The BERT model, which we specifically fine-tuned for this classification job, usually does a pretty good job identifying the right category when the text uses straightforward terms. For instance, if an article talks clearly about the economy or a sports event, BERT is on point.

GPT-2, on the other hand, is working off simple prompts without fine-tuning. As a result, it sometimes guesses the right category when the article strongly hints at something global (like mentioning the United Nations for “World” news), but it’s basically just taking stabs in the dark for trickier examples. It doesn’t have the specialized training that BERT does, so it’s not as consistent.

**What They Get Right:**

BERT: Does well when the article leaves no room for doubt—like when “stocks” or “Wall Street” show up, it’s obviously “Business.”

GPT-2: Occasionally hits the mark when the text makes the answer super obvious, especially for high-level categories like “World.”

**What They Get Wrong:**

BERT: It can get confused when the article mixes clues from different categories. For example, a piece about a sportswear company’s latest earnings report might throw it off—it might focus on “sports” over “business” or vice versa, depending on what stands out most in the text.

GPT-2: Since it’s just guessing based on patterns in language, it often struggles with subtle distinctions. It might label something about new technology as “Business” just because a company name shows up, even though it’s really “Sci/Tech.”

**How to Improve:**

BERT: Try giving it more training data or refining hyperparameters such as increasing the number of epochs Maybe include examples that combine categories so it can learn to pick the main focus better.

GPT-2: The biggest boost would come from fine-tuning it for classification. Also, giving it a few examples (few-shot prompting) might help it understand the task more clearly.

Overall, BERT’s specialized training helps it handle news classification more reliably, while GPT-2’s general knowledge isn’t as suited for this kind of job without extra effort.

## OPTIONAL. BONUS. Problem 4: Improvements

Implement your suggestions for improving the performance. You only need to implement improvements on one model (encoder-only or decoder-only). Describe your method and report the results on the test set.

In [9]:
# TODO

# increasing the train dataset size
train_dataset = encoded_dataset["train"].select(range(4000))
test_dataset = encoded_dataset["test"].select(range(500))

accuracy_metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return accuracy_metric.compute(predictions=predictions, references=labels)

# Increase the number of training epochs from 2 to 4
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=4,
    weight_decay=0.01,
    logging_steps=500,
    save_steps=500,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    logging_dir="./logs"
)

model_name = "bert-base-uncased"
model = BertClassifier(model_name, num_labels=4)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

trainer.train()
metrics = trainer.evaluate(test_dataset)
print("Improved Test Set Performance:", metrics)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.357104,0.880000
2,0.332900,0.346580,0.914000
3,0.332900,0.346066,0.914000
4,0.071500,0.425579,0.906000


Improved Test Set Performance: {'eval_loss': 0.34657981991767883, 'eval_accuracy': 0.914, 'eval_runtime': 3.7495, 'eval_samples_per_second': 133.352, 'eval_steps_per_second': 8.534, 'epoch': 4.0}


No other directions for this assignment, other than what's here and in the "General Directions" section. You have a lot of freedom with this assignment. Don't get carried away. It is expected the results may vary, being better or worse. Graders are not going to run your notebooks. The notebook will be read as a report on how different models were explored. Since you'll be using libraries, the emphasis will be on your ability to communicate your findings.

## Before You Submit...

1. Re-read the general instructions provided above, and
2. Hit "Kernel"->"Restart & Run All".